###### Importing libraries

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from IPython.display import display
import shutil
import re

###### Extracting all actor links from Wikipedia

In [ ]:
urls = ["https://en.wikipedia.org/wiki/List_of_Indian_film_actors", "https://en.wikipedia.org/wiki/List_of_Indian_film_actresses"] 

base = "https://en.wikipedia.org"
actor_urls = []

for i, url in enumerate(urls, 0):    
    source = requests.get(url).text
    soup = BeautifulSoup(source, 'lxml')
    
    divs = soup.find_all('div', class_="div-col columns column-width")
    for div in divs:
        lis = div.findChildren("a", recursive="False")
        for li in lis:
            actor_urls.append((base+li['href'], i))
            
print("Total {} actors got.".format(len(actor_urls)))
display(actor_urls[:5])

###### Download images from Google

In [ ]:
search_url = "https://www.google.com/search?hl=en&site=imghp&tbm=isch&tbs=isz:l&q="

def google_image(name, fn):
    headers= {"User.Agent":"Chrome/80.0.3987.122"}
    url = search_url+name
    
    req= requests.get(url,headers=headers)
    html=req.content
    
    a = requests.get(url).text
    result_url=""
    b=a.find("http://t1.gstatic.com")
    for i in range(b,b+1000):
        if a[i]=='"':
            break
        result_url+=a[i]
    #print(result_url)
    response = requests.get(result_url, stream=True)
    
    file = open("./Images/{}.jpg".format(fn), 'wb')
    
    response.raw.decode_content = True
    shutil.copyfileobj(response.raw, file)
    del response

###### Download images from Wikipedia only

In [ ]:
def download_image(url, fn):
    response = requests.get(url, stream=True)
    
    file = open("./Images/{}.jpg".format(fn), 'wb')
    
    response.raw.decode_content = True
    shutil.copyfileobj(response.raw, file)
    del response

###### Get actor age

In [ ]:
def get_age(query):
    query += " age"
    query = query.replace(' ', '+')
    URL = f"https://google.com/search?q={query}"
    
    headers = {"User.Agent": "Mozilla/5.0 (Linux; Android 7.0; SM-G930V Build/NRD90M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.125 Mobile Safari/537.36"}
    response = requests.get(URL, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "lxml")
    else:
        # print("{} - not found".format(URL))
        return "Unavailable"
    
    div = soup.find("div", class_="BNeawe").find("div", class_="BNeawe")
    if div:
        if len(div.text) < 50:
            return div.text
        else:
            return "Unavailable"
    else:
        return "Unavailable"

###### Get actor heights

In [ ]:
def get_height(query):
    query += " height"
    query = query.replace(' ', '+')
    URL = f"https://google.com/search?q={query}"
    
    headers = {"User.Agent": "Mozilla/5.0 (Linux; Android 7.0; SM-G930V Build/NRD90M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.125 Mobile Safari/537.36"}
    response = requests.get(URL, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "lxml")
    else:
        # print("{} - not found".format(URL))
        return "Unavailable"
    
    div = soup.find('div', class_="BNeawe iBp4i AP7Wnd")
    if div:
        txt = div.text
        if len(txt) < 10:
            return txt
        else:
            return txt.split('.')[0]
    else:
        table = soup.find("table", class_="LnMnt")
        if table:
            trs = table.find_all('tr')
            for tr in trs:
                if tr.find_all('td')[0].text == 'HEIGHT':
                    return tr.find_all('td')[1].text.split('-')[2][1:7]
                    break
        else:
            div = soup.find('div', class_="BNeawe").find('div', class_="BNeawe")
            if div:
                txt = div.text
                if len(txt) > 10:
                    return txt.split(".")[0]
                else:
                    return div.text
            else:
                return "Unavailable"

###### Check if actor is dead or alive

In [ ]:
def is_alive(query):
    query = "is "+ query + " alive"
    query = query.replace(' ', '+')
    URL = f"https://google.com/search?q={query}"
    
    headers = {"User-Agent": "Mozilla/5.0 (Linux; Android 7.0; SM-G930V Build/NRD90M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.125 Mobile Safari/537.36"}
    response = requests.get(URL, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "lxml")
    else:
        # print("{} - not found".format(URL))
        return "Unavailable"
    soup = BeautifulSoup(response.text, "lxml")
    #print(soup.prettify())
    d = soup.find(text="Deceased")
    c = soup.find(text="Cause of death")
    #print(d, c)
    
    if d or c:
        return "No"
    else:
        return "Yes"

#is_alive("Mahesh Babu")

###### Get all information about each actor

In [ ]:
data = {'name':[], 'image':[], 'gender':[], 'age':[], 'height':[], 'description':[]}

for i, url in enumerate(actor_urls, 1):
    #print(actor_url)
    actor_url = url[0]
    gender = url[1]
    
    source = requests.get(actor_url).text
    soup = BeautifulSoup(source, 'lxml')
    
    name = soup.find('h1', id="firstHeading").text
    
    try:
        paras = soup.find('div', class_="mw-parser-output").find_all('p')
        for para in paras:
            txt = para.text
            if len(txt) > 3: 
                txt = re.sub(r'\[.*?\]', '', txt)
                txt = txt.strip()
                data['description'].append(txt)
                #print("\r{}".format(name), end="\r")
                break
    except AttributeError:
        data['description'].append("NaN")
    
    box = soup.find('table', class_="infobox")
    if box:
        img = box.find("a", class_="image")
        if img:
            img = "https:"+img.find_all("img")[0]['src']
            download_image(img, i)
        else:
            google_image(name+' '+box.find('tr').text, i)
    else:
        google_image(name, i)
        
    age = get_age(name)
    
    height = get_height(name)
    
    data["name"].append(name)
    data["image"].append(str(i)+'.jpg')
    data["age"].append(age)
    data["height"].append(height)
    data["gender"].append("Male" if gender == 0 else "Female")
    
    print("{}/{} Actors downloaded. [{}, {}, {}]".format(i, len(actor_urls), name, age, height))

df = pd.DataFrame(data)
df.to_csv('./data.csv', encoding='utf-8', index=False)

###### Check the data downloaded

In [ ]:
df = pd.read_csv('./data.csv')
display(df.head())
print("Total actors got - {}".format(df.shape[0]))

###### Number of actors without each type of information

In [ ]:
ages = df['age'].value_counts()['Unavailable']
heights = df['height'].value_counts()['Unavailable']
des = df['description'].isnull().sum()

print("Total actors: {}\n".format(df.shape[0]))
print('Number of ages unavailable: {}.\nNumber of heights unavailable: {}.\nNumber of descriptions unavailable: {}'.format(ages, heights, des))

In [ ]:
l = []
for name in df['name']:
    l.append(is_alive(name))